In [40]:
# Nathan Tibbetts
# Lab 6 - Cancer Detection

%matplotlib notebook

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread
from os import listdir
from os.path import isfile, join
#from IPython.display import display, Image

TRAIN_PATH = "cancer_data/train"
TEST_PATH = "cancer_data/test"
# Load Data:
train_in_files  = [f for f in listdir(join(TRAIN_PATH, "inputs"))  if isfile(join(join(TRAIN_PATH, "inputs"),  f))]
train_out_files = [f for f in listdir(join(TRAIN_PATH, "outputs")) if isfile(join(join(TRAIN_PATH, "outputs"), f))]
test_in_files   = [f for f in listdir(join(TEST_PATH, "inputs"))   if isfile(join(join(TEST_PATH, "inputs"),   f))]
test_out_files  = [f for f in listdir(join(TEST_PATH, "outputs"))  if isfile(join(join(TEST_PATH, "outputs"),  f))]

assert (train_in_files == train_out_files), "Mismatch in direcory contents between train-in and train-out files!"
assert (test_in_files == test_out_files), "Mismatch in directory contents between test-in and test-out files!"
N = len(train_in_files) # Number of Test images to use.
T = len(test_in_files) # Number of Train images to use.
S = 5 # Mini-batch size - shared for both train and test.
IN_SHAPE = (64, 64, 3) # Used only to scale down the images we work with - change for final trainings!
OUT_SHAPE = (64, 64) # Match back end to front end, but only one channel.

train_neg = [f for f in train_in_files if f[:3] == "neg"]
train_pos = [f for f in train_in_files if f[:3] == "pos"]
test_neg  = [f for f in test_in_files  if f[:3] == "neg"]
test_pos  = [f for f in test_in_files  if f[:3] == "pos"]
assert (len(train_neg) + len(train_pos) == len(train_in_files)), "Not all train files identifiable as 'pos' or 'neg'!"
assert (len(test_neg)  + len(test_pos)  == len(test_in_files)),  "Not all test files identifiable as 'pos' or 'neg'!"

print "Accuracy Baseline is: %f" % ((len(train_pos)/float(len(train_in_files)) * len(test_pos) +
                                    len(train_neg)/float(len(train_in_files)) * len(test_neg)) / len(test_in_files))
#     % we say pos. * num real pos. cases + % we say neg. * num real neg. cases, all / by num. cases to make a %.
 
features      = np.array([resize(imread(join(TRAIN_PATH, "inputs",  f)), IN_SHAPE)
                          for f in train_in_files[:N]],  dtype='float32')
labels        = np.array([resize(imread(join(TRAIN_PATH, "outputs", f)), OUT_SHAPE)
                          for f in train_in_files[:N]], dtype='int32')
test_features = np.array([resize(imread(join(TEST_PATH,  "inputs",  f)), IN_SHAPE)
                          for f in test_in_files[:T]],   dtype='float32')
test_labels   = np.array([resize(imread(join(TEST_PATH,  "outputs", f)), OUT_SHAPE)
                          for f in test_in_files[:T]],  dtype='int32')
# Note: Above we're not using train_out_files or test_out_files because we have asserted they are the same,
#       and if we shuffle them we want them to still be paired.

# plt.imshow(labels[0])

# Whiten our data - zero mean and unit standard deviation
mean = np.mean(features)#, axis=0) # I think the best way to put these on the same page is to scale them the same,
std  = np.std(features)#, axis=0)  #     which is based on the scaling for our features we know about beforehand.
features      = (features      - mean) / std # REMOVED axis=0 ... MAKES SENSE ???
test_features = (test_features - mean) / std
# features = np.transpose(np.reshape(features,(N,3,32,32)),axes=[0,2,3,1])
# features is (N, 1024, 1024, 3), or (batch_size, height, width, channels)

#plt.imshow(features[N-1]*0.1 + 0.5)

HEIGHT = np.shape(features)[1]
WIDTH = np.shape(features)[2]
CHANNELS = np.shape(features)[3]

Accuracy Baseline is: 0.500000


In [41]:
# Layer Defining helper functions

# Convolution Layer:
def conv( x, filter_size=3, stride=2, num_filters=64, is_output=False, name="conv" ):
    '''
    x is an input tensor
    Declare a name scope using the "name" parameter
    Within that scope:
      Create a W filter variable with the proper size
      Create a B bias variable with the proper size
      Convolve x with W by calling the tf.nn.conv2d function
      Add the bias
      If is_output is False,
        Call the tf.nn.relu function
      Return the final op
    ''' 
    with tf.variable_scope(name):
        W = tf.get_variable("W",
                            shape=(filter_size,
                                   filter_size,
                                   x.get_shape().as_list()[3],
                                   num_filters),
                            initializer=tf.contrib.layers.variance_scaling_initializer())
        b = tf.get_variable("b",
                            shape=num_filters,
                            initializer=tf.contrib.layers.variance_scaling_initializer())
        # Notice the input x will be one minibatch at a time,
        #    but right now we're just building a computation graph!
        h = tf.nn.conv2d(x, W, [1,stride,stride,1], padding="SAME", name="h")
        h = tf.nn.bias_add(h, b, name="h")
        if not is_output:
            h = tf.nn.relu(h, name="h_hat")
        return h;        

# Fully-Connected Layer:
def fc( x, out_size=50, is_output=False, name="fc" ):
    '''
    x is an input tensor
    Declare a name scope using the "name" parameter
    Within that scope:
      Create a W filter variable with the proper size
      Create a B bias variable with the proper size
      Multiply x by W and add b
      If is_output is False,
        Call the tf.nn.relu function
      Return the final op
    '''
    with tf.variable_scope(name):
        W = tf.get_variable("W",
                            shape=(out_size,x.get_shape().as_list()[0]),
                            initializer=tf.contrib.layers.variance_scaling_initializer())
        b = tf.get_variable("b",
                            shape=(out_size,1),#out_size,
                            initializer=tf.contrib.layers.variance_scaling_initializer())
        h = tf.matmul(W, x, name="h") + b
        #h = tf.nn.bias_add(h, b, name="h")#(tf.reshape(h,[-1]), b, name="h")
        if not is_output:
            h = tf.nn.relu(h, name="h_hat")
        return h;

In [42]:
# Setting up the Computation Graph

tf.reset_default_graph()

x = tf.placeholder(dtype=np.float32,shape=(S,HEIGHT,WIDTH,CHANNELS),name='x')
y = tf.placeholder(dtype=np.int32,shape=(S,HEIGHT,WIDTH),name='y')
x_test = tf.placeholder(dtype=np.float32, shape=(S,HEIGHT,WIDTH,CHANNELS), name="x_test")
y_test = tf.placeholder(dtype=np.int32, shape=(S,HEIGHT,WIDTH), name="y_test")
#lambda 0.01 - 0.05
h0 = conv(x,stride=1,name='conv1',num_filters=2,is_output=True) # Shape h0 = [S,128,128,2]
#DROPOUT btw layers - Layers not nn, because is wrapper, and has training=True when not test
#h1 = conv(h0,stride=1,name='conv2')
#h2 = conv(h1,stride=2,name='conv3_stride2') # Shape h2 = [S,16,16,64]
#h3 = conv(h2,stride=1,name='conv4')
#h4 = conv(h3,stride=2,name='conv5_stride2',num_filters=48) # Shape h4 = [S,8,8,48]
#h5 = conv(h4,stride=2,name='conv6_stride2',num_filters=48) # Shape h5 = [S,4,4,48]
#h6 = conv(h5,stride=1,name='conv7',is_output=True,num_filters=32) # Shape h6 = [S,4,4,32]

shape1 = h0.get_shape().as_list()#h6.get_shape().as_list()
# flat1 = tf.transpose(tf.reshape(h6,[shape1[0],shape1[1]*shape1[2]*shape1[3]]))
# flat1 is [512,S]
# fc1 = fc(flat1,out_size=64,name='fc1')# fc1 is [64,S]
# fc2 = fc(fc1,out_size=10,name='fc2',is_output=True) # fc2 is [10,S]

#h1b = conv(h0,stride=2,name='conv2b') # Shape h1b -> [S,16,16,64]
#h2b = conv(h1b,stride=2,name='conv3b') # Shape h2b -> [S,8,8,64]
#h3b = conv(h2b,stride=2,name='conv4b',is_output=True) # Shape h3b -> [S,4,4,64]
#shape2 = h3b.get_shape().as_list()
#flat2 = tf.transpose(tf.reshape(h3b,[shape2[0],shape2[1]*shape2[2]*shape2[3]]))
## flat2 is [1024,S]
#fc1b = fc(flat2,out_size=32)#,is_output=True)
## fc1b is [32,S]
#h6 = tf.concat([fc1,fc1b],axis=0,name='combined')
#fc2 = fc(h6,out_size=10,name='fc2',is_output=True) # fc2 is [10,S]

with tf.name_scope(name="train_loss") as scope:
    xent = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y,
        logits=h0,
        name='cross_entropy')
    loss = tf.reduce_mean(xent)
    
#with tf.name_scope(name="test_loss") as scope:
#    test_xent = tf.nn.sparse_softmax_cross_entropy_with_logits(
#        labels=y,
#        logits=tf.transpose(fc2),
#        name='cross_entropy')
#    test_loss = tf.reduce_mean(xent)

with tf.name_scope(name="accuracy") as scope:
    prediction = tf.cast(tf.argmax(h0,axis=-1),tf.int32)#tf.one_hot(tf.argmax(fc2,axis=0),10)
    #acc = tf.reduce_sum(tf.abs(y-prediction),name='accuracy')#1-(0.5*(1.0/S)*....)
    correct_prediction = tf.equal(prediction,y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    #accuracy = tf.metrics.accuracy(y,prediction,name="acc")
    
#with tf.name_scope(name="test_accuracy") as scope:
#    test_prediction = tf.argmax(h0,axis=0)
#    test_correct_prediction = tf.equal(prediction,y)
#    test_accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

c = 0.001 # Learning Rate
train_step = tf.train.AdamOptimizer(learning_rate=c,name='adam_optimizer').minimize(xent)
#test_step = tf.train.AdamOptimizer(learning_rate=0,name='adam_tester').minimize(test_xent)


#xx = features[:10]
#yy = labels[:10]

with tf.variable_scope("train"):
    tf.summary.scalar('xent_loss', loss)
    tf.summary.scalar('accuracy', accuracy)
    train_merged = tf.summary.merge_all()

with tf.variable_scope("test"):
    tf.summary.scalar('xent_loss', loss)
    tf.summary.scalar('accuracy', accuracy)
    test_merged = tf.summary.merge_all()

In [43]:
# Training

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    train_writer = tf.summary.FileWriter("./tf_logs", sess.graph)
    
    #for t in xrange(S,100*S,S):#(S,D+S,S):
    #for t in np.ones(100,dtype=np.int32)*32:
    for t in xrange(N/S):
    #for t in np.ones(15,dtype=np.int32): # Overfitting test
        ix = np.random.randint(N,size=S)
        _, loss_value, acc, graph_summary = sess.run(
            [train_step, loss, accuracy, train_merged],
            #feed_dict={x:features[S*t:S*(t+1)],y:labels[S*t:S*(t+1)]})
            #feed_dict={x:[features[n] for n in np.random.randint(N,size=S)],
            #           y:[labels[n]   for n in np.random.randint(N,size=S)]})
            feed_dict={x:features[ix],y:labels[ix]})
        train_writer.add_summary(graph_summary, t)
        if t%S == 0:
            ix = np.random.randint(T,size=S)
            tt = t/S
            test_loss_value, test_acc, test_summary = sess.run(
                [loss, accuracy, test_merged],
                #feed_dict={x:test_features[S*tt:S*(tt+1)], y:test_labels[S*tt:S*(tt+1)]})
                feed_dict={x:test_features[ix],y:test_labels[ix]})
            train_writer.add_summary(test_summary, t)
        print t, loss_value, acc
        #print test_loss, test_acc
        
    train_writer.close()

0 0.853242 0.685059
1 1.02838 0.588623
2 1.65003 0.637451
3 0.643704 0.714746
4 0.681202 0.80415
5 1.11801 0.560303
6 2.03151 0.535449
7 0.862871 0.64502
8 0.676371 0.736719
9 0.946723 0.672412
10 0.939861 0.547754
11 0.778197 0.686377
12 0.874903 0.593603
13 1.11012 0.576514
14 0.826635 0.757129
15 0.921193 0.617676
16 0.903186 0.684668
17 0.927528 0.677734
18 0.649145 0.782812
19 0.810358 0.726074
20 0.945107 0.653223
21 0.66761 0.73667
22 1.02317 0.668555
23 1.11206 0.589111
24 0.666395 0.723096
25 0.943271 0.583838
26 1.20785 0.5375
27 0.793616 0.696094
28 0.809882 0.5604
29 0.905668 0.654932
30 1.03492 0.610986
31 1.08529 0.442578
32 0.84592 0.651221
33 1.38487 0.470654
34 1.17056 0.49502
35 0.878932 0.698193
36 0.759284 0.662256
37 0.702994 0.789551
38 0.923797 0.634863
39 0.830039 0.678467
40 1.14797 0.467334
41 0.901823 0.603076
42 0.934293 0.560107
43 1.20206 0.59502
44 0.889643 0.655908
45 1.21429 0.574219
46 1.10628 0.621191
47 0.767284 0.729102
48 0.81712 0.652051
49 0.8474

In [ ]:
# import PIL.Image as PILImage
# from IPython.display import display, Image

# display(PILImage.open('lab5_comp_graph'))
# display(PILImage.open('lab5_graphs.png'))